Extract the data from https://s3.amazonaws.com/tripdata/index.html for the year 2022

In [8]:
import os
import requests
import zipfile

URLs = ['https://s3.amazonaws.com/tripdata/JC-202201-citibike-tripdata.csv.zip',\
       'https://s3.amazonaws.com/tripdata/JC-202202-citibike-tripdata.csv.zip',\
       'https://s3.amazonaws.com/tripdata/JC-202203-citibike-tripdata.csv.zip',\
        'https://s3.amazonaws.com/tripdata/JC-202204-citibike-tripdata.csv.zip',\
        'https://s3.amazonaws.com/tripdata/JC-202205-citibike-tripdata.csv.zip',\
        'https://s3.amazonaws.com/tripdata/JC-202206-citibike-tripdata.csv.zip',\
        'https://s3.amazonaws.com/tripdata/JC-202207-citbike-tripdata.csv.zip',\
        'https://s3.amazonaws.com/tripdata/JC-202208-citibike-tripdata.csv.zip',\
        'https://s3.amazonaws.com/tripdata/JC-202209-citibike-tripdata.csv.zip',\
        'https://s3.amazonaws.com/tripdata/JC-202210-citibike-tripdata.csv.zip',\
        'https://s3.amazonaws.com/tripdata/JC-202211-citibike-tripdata.csv.zip',\
        'https://s3.amazonaws.com/tripdata/JC-202212-citibike-tripdata.csv.zip']

# Create a directory to store the downloaded files
os.makedirs('citibike-data', exist_ok=True)

for url in URLs:
    r = requests.get(url)
    with open('citibike-data/' + url.split('/')[-1], 'wb') as f:
        f.write(r.content)

# Extract all the zip files in the directory
for file in os.listdir('citibike-data'):
    if file.endswith('.zip'):
        with zipfile.ZipFile(os.path.join('citibike-data', file), 'r') as zip_ref:
            zip_ref.extractall('citibike-data')


In [1]:
import pandas as pd
import glob

# Create a list of CSV file paths
csv_files = glob.glob("citibike-data/*.csv")

# Create an empty list to store dataframes
dfs = []

# Iterate over the list of CSV file paths
for file in csv_files:
    # read the CSV file into a dataframe
    df = pd.read_csv(file)
    # append the dataframe to the list of dataframes
    dfs.append(df)

# Concatenate the list of dataframes into a single dataframe
df = pd.concat(dfs)

# Print the first five rows of the dataframe
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,CA5837152804D4B5,electric_bike,2022-01-26 18:50:39,2022-01-26 18:51:53,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member
1,BA06A5E45B6601D2,classic_bike,2022-01-28 13:14:07,2022-01-28 13:20:23,Essex Light Rail,JC038,Essex Light Rail,JC038,40.712774,-74.036486,40.712774,-74.036486,member
2,7B6827D7B9508D93,classic_bike,2022-01-10 19:55:13,2022-01-10 20:00:37,Essex Light Rail,JC038,Essex Light Rail,JC038,40.712774,-74.036486,40.712774,-74.036486,member
3,6E5864EA6FCEC90D,electric_bike,2022-01-26 07:54:57,2022-01-26 07:55:22,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member
4,E24954255BBDE32D,electric_bike,2022-01-13 18:44:46,2022-01-13 18:45:43,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member


In [2]:
df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

In [3]:
#Split the date and time values(ISO 8601 timestamp,YYYY-MM-DD HH:MM:SS ) in columns'started_at' & 'ended_at' to separate the dates & times
# and name the new columns 'started_at_date', 'started_at_time' &'ended_at_date', 'ended_at_time'

import re
# Use apply() with lambda function to extract date and time components
date_regex = r'^(\d{4}-\d{2}-\d{2}) (\d{2}:\d{2}:\d{2})$'

df[['started_at_date', 'started_at_time']] = df['started_at'].apply(lambda x: pd.Series(re.match(date_regex, x).groups()) if re.match(date_regex, x) else pd.Series([None, None]))
df[['ended_at_date', 'ended_at_time']] = df['ended_at'].apply(lambda x: pd.Series(re.match(date_regex, x).groups()) if re.match(date_regex, x) else pd.Series([None, None]))

df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,started_at_date,started_at_time,ended_at_date,ended_at_time
0,CA5837152804D4B5,electric_bike,2022-01-26 18:50:39,2022-01-26 18:51:53,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member,2022-01-26,18:50:39,2022-01-26,18:51:53
1,BA06A5E45B6601D2,classic_bike,2022-01-28 13:14:07,2022-01-28 13:20:23,Essex Light Rail,JC038,Essex Light Rail,JC038,40.712774,-74.036486,40.712774,-74.036486,member,2022-01-28,13:14:07,2022-01-28,13:20:23
2,7B6827D7B9508D93,classic_bike,2022-01-10 19:55:13,2022-01-10 20:00:37,Essex Light Rail,JC038,Essex Light Rail,JC038,40.712774,-74.036486,40.712774,-74.036486,member,2022-01-10,19:55:13,2022-01-10,20:00:37
3,6E5864EA6FCEC90D,electric_bike,2022-01-26 07:54:57,2022-01-26 07:55:22,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member,2022-01-26,07:54:57,2022-01-26,07:55:22
4,E24954255BBDE32D,electric_bike,2022-01-13 18:44:46,2022-01-13 18:45:43,12 St & Sinatra Dr N,HB201,12 St & Sinatra Dr N,HB201,40.750604,-74.024020,40.750604,-74.024020,member,2022-01-13,18:44:46,2022-01-13,18:45:43


In [17]:
import numpy as np
from scipy import stats
# check for missing values
print(df.isnull().sum())

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
started_at_date       0
started_at_time       0
ended_at_date         0
ended_at_time         0
dtype: int64


In [18]:
# drop the rows with missing values
df.dropna(inplace=True)

In [19]:
# check for duplicate rows
duplicates = df.duplicated()

# count the number of duplicate rows
num_duplicates = duplicates.sum()

# print the result
print("Number of duplicate rows: ", num_duplicates)

Number of duplicate rows:  0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 892281 entries, 0 to 48584
Data columns (total 17 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             892281 non-null  object 
 1   rideable_type       892281 non-null  object 
 2   started_at          892281 non-null  object 
 3   ended_at            892281 non-null  object 
 4   start_station_name  892281 non-null  object 
 5   start_station_id    892281 non-null  object 
 6   end_station_name    892281 non-null  object 
 7   end_station_id      892281 non-null  object 
 8   start_lat           892281 non-null  float64
 9   start_lng           892281 non-null  float64
 10  end_lat             892281 non-null  float64
 11  end_lng             892281 non-null  float64
 12  member_casual       892281 non-null  object 
 13  started_at_date     892281 non-null  object 
 14  started_at_time     892281 non-null  object 
 15  ended_at_date       892281 non-null

In [21]:
# remove trailing spaces in column names
df.columns = df.columns.str.strip()
# save the modified dataframe to a new CSV file
df.to_csv("citibike_tripdata_2022.csv", index=False)

# print a message to confirm that the file was saved
print("File saved successfully.")

File saved successfully.
